## Loading Serialized Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.losses import BinaryFocalCrossentropy
from keras.utils import register_keras_serializable
from keras.activations import swish
from keras.layers import Dropout

@register_keras_serializable()
class FocalDiceLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.25, gamma=2., name='focal_dice_loss'):
        super(FocalDiceLoss, self).__init__(name=name)
        self.alpha = alpha
        self.gamma = gamma
        self.focal = BinaryFocalCrossentropy(from_logits=False, alpha=self.alpha, gamma=self.gamma)
        
    def call(self, y_true, y_pred):
        focal_loss = self.focal(y_true, y_pred)
        dice = dice_loss(y_true, y_pred)
        return focal_loss + dice

@register_keras_serializable()
class FixedDropout(Dropout):
    def call(self, inputs, training=None):
        return super().call(inputs, training=True)

In [ ]:
# specificity (true negative rate)
def specificity(y_true, y_pred, smooth=1e-7):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    tn = tf.reduce_sum((1 - y_true) * (1 - y_pred))
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    return tn / (tn + fp + smooth)

# F1 score (Dice coefficient)
def f1_score(y_true, y_pred, smooth=1e-7):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)

    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))

    precision = tp / (tp + fp + smooth)
    recall = tp / (tp + fn + smooth)
    f1 = 2 * precision * recall / (precision + recall + smooth)
    return f1

# Matthew's correlation coefficient
def mcc(y_true, y_pred, smooth=1e-7):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)

    tp = tf.reduce_sum(y_true * y_pred)
    tn = tf.reduce_sum((1 - y_true) * (1 - y_pred))
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))

    numerator = (tp * tn) - (fp * fn)
    denominator = tf.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return numerator / (denominator + smooth)

In [ ]:
# load serialized model
model = tf.keras.models.load_model(
    'efficientnetb0_unet.keras',
    custom_objects={'swish': swish, 'FixedDropout': FixedDropout, 'focal_dice_loss': FocalDiceLoss()},
    compile=False,
)

# define optimizer
adam_optimizer = keras.optimizers.Adam(
    learning_rate=1e-4,
    name='adam',
    beta_1=0.9,
    beta_2=0.999,
)

# evaluation metrics
accuracy = tf.keras.metrics.BinaryAccuracy(name='accuracy')
iou = tf.keras.metrics.BinaryIoU(target_class_ids=(0, 1), name='iou')
sensitivity = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()

# compile U-Net model with defined optimizer, loss and metrics
model.compile(
    optimizer=adam_optimizer,
    loss=FocalDiceLoss(alpha=0.25, gamma=2.0),
    metrics=[accuracy, iou, sensitivity, precision, specificity, f1_score, mcc],
)